# Coin Recognition Program

I'm making a program for coin recognition using Python and OpenCV. The first step was installing the OpenCV library with the command:

In [ ]:
pip install opencv -python

After installing and setting up OpenCV, it was necessary to collect images of coins. The coins were photographed on a white background for easier extraction. The photographed images are saved in the 'obrada' folder where the Python programs will also be located. First, an explanation of the 'obrada1.py' file follows. Importing two essential libraries in Python for numerical computing and computer vision tasks:

In [3]:
import numpy as np
import cv2

Importing image, resizing it and making a copy that will be used later:

In [4]:
slika = cv2.imread("slika5.jpg")
slika = cv2.resize(slika, (640, 800))
s_kopija = slika.copy()

Next step is applying a Gaussian blur to an image, that is reducing image noise and detail through image smoothing. The parameters include the input image 'slika5.jpg', the size of the Gaussian kernel (7, 7) for blurring, and the standard deviation 3, determining the extent of blur applied in both the X and Y directions:

In [5]:
slika = cv2.GaussianBlur(slika, (7, 7), 3)

Following code converts the image from the BGR color space to grayscale. Then, it applies a binary thresholding operation to the grayscale image, setting pixel values below 138 to 0 and above 138 to 255, creating a binary image where objects of interest are highlighted (The parameters need to be adjusted for each specific image):

In [ ]:
gray = cv2.cvtColor(slika, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 138, 255, cv2.THRESH_BINARY)

This code segment uses the cv2.findContours() function to identify contours in the thresholded binary image thresh, employing a hierarchical retrieval mode and retaining all contour points without approximation. It then iterates over each contour found, calculates its area using cv2.contourArea(), and stores the contour index and its corresponding area in a dictionary named area:

In [ ]:
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
area = {}
for i in range(len(contours)):
    cnt = contours[i]
    ar = cv2.contourArea(cnt)
    area[i] = ar

This code sorts the dictionary area based on contour areas in descending order, converting it into a numpy array rez. Then, it counts the number of contours with areas greater than 500 pixels by identifying indices where the second column of rez (contour areas) exceeds 500 using np.argwhere(), and computing its shape to determine the count:

In [ ]:
srt = sorted(area.items(), key=lambda x: x[1], reverse=True)
rez = np.array(srt).astype("int")
num = np.argwhere(rez[:, 1] > 500).shape[0]

This code iterates over the sorted contours, drawing each contour on a copy of the image s_kopija and finding the minimum enclosing circle around each contour using cv2.minEnclosingCircle(). It calculates the diameter of each coin by doubling the radius, appends the diameter and contour index to the list diameters, providing a list of tuples containing the diameter and contour index for each identified coin:

In [ ]:
diameters = []
for i in range(1, num):
    s_kopija = cv2.drawContours(s_kopija, contours, rez[i, 0], (0, 255, 0), 3)
    (x, y), radius = cv2.minEnclosingCircle(contours[rez[i, 0]])
    diameter = radius * 2  
    diameters.append((diameter, rez[i, 0]))

This code segment sorts the list of coin diameters in descending order based on their diameter values. It then defines the values of the coins based on the sorted diameters, assuming there are exactly three types of coins with values of 5.00, 1.00, and 0.05, respectively. Finally, it initializes a variable total_value to calculate the total value of the coins (assuming we have exactly 3 types of coins):

In [ ]:
diameters.sort(reverse=True, key=lambda x: x[0])
coin_values = [5.00, 1.00, 0.05]  
total_value = 0

This code iterates over the sorted list of coin diameters and their corresponding contour indices. It assigns a value to each coin based on its position in the sorted list, cycling through the predefined coin values if more coins are detected than predefined values. The total value of the coins is calculated incrementally. It prints the diameter and assigned value for each recognized coin and annotates the image with the value of the coin using OpenCV's cv2.putText() function:

In [ ]:
for idx, (diameter, contour_idx) in enumerate(diameters):
    coin_value = coin_values[idx % len(coin_values)]  
    total_value += coin_value
    
    
    print(f"Recognized coin {idx+1}: Diameter = {diameter:.2f} pixels, Value = {coin_value} KM")
    
    
    (x, y), radius = cv2.minEnclosingCircle(contours[contour_idx])
    center = (int(x), int(y))
    cv2.putText(s_kopija, f"{coin_value} KM", center, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

Printing total value and showing final results:

In [ ]:
print("Total value:", total_value, "KM")

cv2.imshow("rezultat", s_kopija)
cv2.waitKey(0)
cv2.destroyAllWindows()

For the file obrada.py, the same principle was used, only there are multiple types of different coins in the image. In this example, we can notice how the parameters had to be adjusted:

In [ ]:
ret, thresh = cv2.threshold(gray, 142, 255, cv2.THRESH_BINARY)

For file 'obrada2.py': This code snippet iterates through the contours of coins detected in the image. For each contour, it calculates the diameter based on the minimum enclosing circle. Depending on the diameter range, it assigns a corresponding coin value. Contours with recognized coin values are grouped together based on their assigned values in the coin_groups dictionary:

In [ ]:
coin_groups = {}

for contour in contours:
    (x, y), radius = cv2.minEnclosingCircle(contour)
    diameter = radius * 2  

    if 75 <= diameter <= 76:
        coin_value = 0.05
    elif 88 <= diameter <= 89:
        coin_value = 0.1
    elif 90 <= diameter <= 91:
        coin_value = 0.2
    elif 95 <= diameter <= 97.8:
        coin_value = 1
    elif 98 <= diameter <= 99:
        coin_value = 0.5
    elif 100 <= diameter <=120:
        coin_value = 5
    else:
        coin_value = None

    if coin_value is not None:
        coin_groups.setdefault(coin_value, []).append(contour)

The coin_values dictionary maps coin values to their corresponding denominations in the currency of KM (Konvertibilna Marka). Each key represents a coin value (in KM), while the corresponding value is a string indicating the denomination of the coin:

In [ ]:
coin_values = {
    5.00: "5 KM",
    1.00: "1 KM",
    0.50: "0.50 KM",
    0.20: "0.20 KM",
    0.10: "0.10 KM",
    0.05: "0.05 KM"
}

The grouped_coins dictionary organizes detected coins based on their values. It iterates over each coin value in the coin_groups dictionary and assigns the corresponding denomination from the coin_values dictionary. The contours of coins belonging to each denomination are then grouped together under their respective denomination in the grouped_coins dictionary:

In [ ]:
grouped_coins = {}
for coin_value, group in coin_groups.items():
    grouped_coins.setdefault(coin_values[coin_value], []).extend(group)

This code segment iterates over the grouped_coins dictionary, printing the denomination group and the count of coins in each group. Then, it calculates the total value of all the coins detected in KM (Konvertibilna Marka) by summing up the product of each coin value and the count of coins in that value group. Finally, it prints the total value in KM:

In [ ]:
for group, coins in grouped_coins.items():
    print(f"Group: {group}, Count: {len(coins)}")


total_value = sum(coin_value * len(coins) for coin_value, coins in coin_groups.items())
print("Total value in KM:", total_value)

This code iterates over the items in the grouped_coins dictionary. For each group of coins, it generates a random color using NumPy and draws contours of the coins onto the image s_kopija using OpenCV's cv2.drawContours() function. Each group of coins is drawn with contours of the same color to visually distinguish them on the image:

In [ ]:
for group, coins in grouped_coins.items():
    color = np.random.randint(0, 255, size=3).tolist() 
    for contour in coins:
        s_kopija = cv2.drawContours(s_kopija, [contour], -1, color, 3)